In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import os
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

In [6]:
input_dim_h = 512
input_dim_w = 512

transform = transforms.Compose([
                    transforms.Resize((input_dim_w, input_dim_h)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

notebook_path = os.path.abspath("Custom_CNN.ipynb")
data_path = os.path.dirname(notebook_path) + '/data/columbia-prcg-datasets/'

full_set = datasets.ImageFolder(root=data_path, transform=transform)
train_size = int(0.8 * len(full_set))
test_size = len(full_set) - train_size
train_set, test_set = torch.utils.data.random_split(full_set, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, num_workers=0)

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),  # (1024, 1024, 3) -> (32, 32, 32)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  # (512, 512, 32)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),  # (16, 16, 64)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  #  (256, 256, 64)
        # self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(128 * 128 * 64, 1)
        # self.fc1 = nn.Linear(128 * 128 * 64, 1000)
        # self.fc2 = nn.Linear(1000, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        # out = self.drop_out(out)
        out = self.fc1(out)
        # out = self.fc2(out)
        out = self.sigmoid(out)
        return out

model = ConvNet()

In [11]:
import torch.optim as optim

# Define the loss function and optimizer
loss_fn = torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [20]:
for epoch in range(1):
    # Train the model
    running_loss = 0.0
    model.train()
    for i, [inputs, labels] in enumerate(train_loader):   
        outputs = model(inputs)
        loss = loss_fn(outputs, labels.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()

        if i % 10 == 1:
            print (running_loss / i)
        
    # Test current version of model to obtain accuracy    
    correct = total = 0 
    with torch.no_grad():
        for [inputs, labels] in test_loader:   
            probs = model(inputs)
            t = torch.Tensor([0.5])  # 0.5 acts as threshold
            outputs = (probs > t).float() * 1
            total += len(outputs)
            correct += (outputs == labels.float()).float().sum()
            
    accuracy = correct / total
    print ('Loss: {}     Accuracy: {}'.format(running_loss,  accuracy))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
0.0
10.047644181685014
15.78915514264788
16.935142209452966
14.8264017802913
15.169972588034238
13.136059401465245
13.231756774472519
13.644948888708043
13.360054351471282
13.678723514670192
13.44211856738941
13.016266326273769
13.710048835696155
13.325599021099983
13.175056609096906
12.871593866289032
13.24996352614018
13.1285516517597
13.019853062654665
12.921970253560081
12.96431814437794
13.002833646886488
13.396858908913352
13.528881896086254
13.760468874319615
13.762577758438285
13.968449983209702
13.963007305864762
13.862986731775027
14.136801697487055
14.037625070553501
14.03070562130937
13.940726859331852
13.774996055535

In [21]:
correct = total = 0 
with torch.no_grad():
    for [inputs, labels] in test_loader:   
        probs = model(inputs)
        t = torch.Tensor([0.5])  # 0.5 acts as threshold
        outputs = (probs > t).float() * 1
        total += len(outputs)
        correct += (outputs == labels.float()).float().sum()

accuracy = correct / total

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 52 %


In [0]:
print (len(train_loader))
print (len(test_loader))

1280
320
